In [79]:
import numpy as np
import pandas as pd
import h5py

train_df = pd.read_csv('data/training_set.csv')

In [3]:
train_df.head()

,object_id,mjd,passband,flux,flux_err,detected
0,615,59750.4229,2,-544.810303,3.622952,1
1,615,59750.4306,1,-816.434326,5.553370,1
2,615,59750.4383,3,-471.385529,3.801213,1
3,615,59750.4450,4,-388.984985,11.395031,1
4,615,59752.4070,2,-681.858887,4.041204,1


In [4]:
train_df.groupby(['object_id', 'passband'])['flux'].count().head()

object_id  passband
615        0           63
           1           58
           2           58
           3           58
           4           58
Name: flux, dtype: int64

In [13]:
toids = np.load('data/test_obj_ids.npy')

In [14]:
a=np.unique(toids)

In [23]:
import tqdm

prev = -1
curr = -1
for oid in tqdm.tqdm(toids, total=toids.size):
    curr = oid
    if prev > curr:
        print(f'Alarm : prev is {prev} and curr is {curr}')
    prev = curr



  0%|          | 0/453653104 [00:00<?, ?it/s]

  0%|          | 209183/453653104 [00:00<03:38, 2077832.15it/s]

  0%|          | 434446/453653104 [00:00<03:29, 2164743.24it/s]

  0%|          | 668441/453653104 [00:00<03:23, 2221829.75it/s]

  0%|          | 906572/453653104 [00:00<03:20, 2261291.51it/s]

  0%|          | 1151040/453653104 [00:00<03:16, 2297889.97it/s]

  0%|          | 1416292/453653104 [00:00<03:11, 2357643.72it/s]

  0%|          | 1687075/453653104 [00:00<03:07, 2407565.76it/s]

  0%|          | 1958976/453653104 [00:00<03:04, 2446244.33it/s]

  0%|          | 2236908/453653104 [00:00<03:01, 2484025.78it/s]

  1%|          | 2513543/453653104 [00:01<02:59, 2511602.25it/s]

  1%|          | 2794055/453653104 [00:01<02:57, 2537909.87it/s]

  1%|          | 3077821/453653104 [00:01<02:55, 2563198.69it/s]

  1%|          | 3365491/453653104 [00:01<02:54, 2587174.83it/s]

  1%|          | 3653927/453653104 [00:01<02:52, 2608341.02it/s]

  1%|          | 3944726/453653

KeyboardInterrupt: 

In [8]:
bug = 99406585
w=1
for p in toids[bug-w:bug+w]:
    print(p)
    
toids[bug]

130788054
1000183


1000183

In [9]:
sorted_toids = np.sort(toids, kind='heapsort')

In [50]:
toids1 = toids[:bug]
toids2 = toids[bug:]

In [62]:
toids1[40:5000]

array([ 13,  13,  13, ..., 116, 116, 116])

In [63]:
toids2[50:2546]

array([1000183, 1000183, 1000183, ..., 1000946, 1000946, 1000946])

In [53]:
u1 = np.unique(toids1)
u2 = np.unique(toids2)

In [56]:
common = np.intersect1d(u1,u2,assume_unique=True)

In [57]:
common

array([], dtype=int64)

In [68]:
np.intersect1d(u1,u2,assume_unique=True)

array([130787974])

In [69]:
demo_test = pd.read_csv('data/test_set.csv.zip', nrows=10e6)

In [77]:
demo_test.memory_usage()/1e6

Index         0.00008
object_id    80.00000
mjd          80.00000
passband     80.00000
flux         80.00000
flux_err     80.00000
detected     80.00000
dtype: float64

In [83]:
demo_test = pd.read_csv('data/test_set.csv.zip', nrows=10)

In [84]:
demo_test.head()

,object_id,mjd,passband,flux,flux_err,detected
0,13,59798.3205,2,-1.299735,1.357315,0
1,13,59798.3281,1,-2.095392,1.148654,0
2,13,59798.3357,3,-0.923794,1.763655,0
3,13,59798.3466,4,-4.009815,2.602911,0
4,13,59798.3576,5,-3.403503,5.367328,0


In [85]:
dd = pd.read_csv('data/test_set.csv.zip', nrows=2, skiprows=2)
dd.head()

,13,59798.3281,1,-2.095392,1.148654,0
0,13,59798.3357,3,-0.923794,1.763655,0
1,13,59798.3466,4,-4.009815,2.602911,0


In [248]:
# Get accurate splits for toids ie. no user leakage between splits

def get_splits(oids, nsplits):
    '''
    Given a numpy array containing sequential object ids, not necessarily sorted, will return via 'final_splits'
    the indexes where splitting occurs, each index indicating the first element of the chunk, as well as number of
    elements in chunk via 'nrows_per_split'
    '''
    rough_splits = np.round(np.linspace(0, oids.size, nsplits)[:-1],0).astype(int)

    # Refine splits
    final_splits = []
    for s in rough_splits:
        p_id = oids[s]
        n_id = oids[s]
        ind = s
        while n_id == p_id:
            p_id = n_id
            ind -= 1
            n_id = oids[ind]
        final_splits.append(ind+1)
    final_splits.append(oids.size) # Not size-1 to account for global  header
    
    final_splits = [s+1 for s in final_splits] # +1 to jump global header
    
    nrows_per_split = [final_splits[i+1]-final_splits[i] for i in np.arange(len(final_splits)-1)]
    
    return final_splits, nrows_per_split

final_splits = get_splits(toids, 15)

In [257]:
# Test splitting method on train data first

# get data oids
doids = pd.read_csv('data/training_set.csv', usecols=['object_id']).values

In [258]:
dsplits, dchunksizes = get_splits(doids, 15)

In [250]:
save_dir = './data/train_chunks/'
chunk_name = 'training_set_chunk'
source_csv = './data/training_set.csv'
header = ['object_id', 'mjd', 'passband', 'flux', 'flux_err', 'detected']

for i, (chunk_start_index, chunk_size) in enumerate(zip(dsplits, dchunksizes)):   
    csv_chunk = pd.read_csv(source_csv, skiprows=chunk_start_index, nrows=chunk_size, header=None, names=header)
    csv_chunk.to_hdf(save_dir+chunk_name+str(i)+'.h5', key='w', mode='w')

In [251]:
# Read train chunks
import os
cs = []
cnames = os.listdir(save_dir)
for chunk in cnames:
    cs.append(pd.read_hdf(save_dir + chunk))

In [252]:
recon_train = pd.concat(cs, axis=0).sort_values(['object_id', 'mjd']) #Checked OK